In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [6]:
import joblib
import pickle

In [ ]:
#import joblib
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

def extract_hand_landmarks(image):
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_hand_landmarks:
        for index, landmarks in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Draw bounding box
            lmx = [landmark.x for landmark in landmarks.landmark]
            lmy = [landmark.y for landmark in landmarks.landmark]
            min_x, max_x, min_y, max_y = int(min(lmx)*image.shape[1]), int(max(lmx)*image.shape[1]), int(min(lmy)*image.shape[0]), int(max(lmy)*image.shape[0])
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            
            # Indicate left or right hand
            hand_label = results.multi_handedness[index].classification[0].label
            hand_label = "Right" if hand_label == "Left" else "Left"
            cv2.putText(image, hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks_array = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
        return landmarks_array
    return None

def landmarks_to_heatmap(landmarks, img_size=(128, 128), blob_size=5):
    heatmap = np.zeros(img_size)
    for landmark in landmarks:
        x, y = int(landmark[0] * img_size[0]), int(landmark[1] * img_size[1])
        cv2.circle(heatmap, (x, y), blob_size, (255, 255, 255), -1)
    return heatmap

def preprocess_image(img, img_size=(128, 128)):
    landmarks = extract_hand_landmarks(img)
    if landmarks:
        heatmap = landmarks_to_heatmap(landmarks, img_size)
        return heatmap / 255.0
    return None
feature_extractor = load_model("cnn_model_3_mediapipe_90.h5")
dt_classifier = joblib.load('dt_classifier_3_mediapipe_90.pkl')
unique_labels = joblib.load('unique_labels_3_mediapipe_90.pkl')
img_size=(128,128)
#def real_time_prediction(n):
def real_time_prediction(model, feature_extractor):
       # Initialize webcam
    cap = cv2.VideoCapture(0)

    while True:
    # Capture frame-by-frame
        ret, frame = cap.read()
        scale_factor = 1.5  # Adjust this value as needed
        new_width = int(frame.shape[1] * scale_factor)
        new_height = int(frame.shape[0] * scale_factor)
        resized_frame = cv2.resize(frame, (new_width, new_height))

    # Check if frame captured successfully
        if not ret:
            break

    # Preprocess the frame
        processed_img = preprocess_image(resized_frame, img_size)
    
        if processed_img is not None:
            processed_img_reshaped = processed_img.reshape((-1, img_size[0], img_size[1], 1))
        
        # Extract features from CNN
            features = feature_extractor.predict(processed_img_reshaped)
        
        # Predict using the Decision Tree classifier
            prediction = dt_classifier.predict(features)
            
            # Convert integer label back to string label
            predicted_gesture = unique_labels[prediction[0]]
        
        # Display the result on the frame
        # Instead of a fixed position, compute the position dynamically
            text_position = (int(resized_frame.shape[1] * 0.05), int(resized_frame.shape[0] * 0.05))

            cv2.putText(resized_frame, predicted_gesture, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)

    # Display the frame
        cv2.imshow('Gesture Recognition', resized_frame)

    # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the capture
    cap.release()
    cv2.destroyAllWindows()
#real_time_prediction(5)
real_time_prediction(dt_classifier, feature_extractor)

In [1]:
from tensorflow.keras.models import load_model
import cv2
import mediapipe as mp
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [2]:
import joblib
import pickle

In [3]:
import os

In [8]:
from flask import Flask, render_template, Response, jsonify

In [ ]:

app = Flask(__name__)

# Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

'''
def extract_hand_landmarks(image):
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks_array = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
        return landmarks_array
    return None'''
last_landmarks = None
last_bounding_box = None
last_hand_label = None
def extract_hand_landmarks(image):
    global last_landmarks, last_bounding_box, last_hand_label
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if results.multi_hand_landmarks:
        for index, landmarks in enumerate(results.multi_hand_landmarks):
            # Draw landmarks
            mp_drawing.draw_landmarks(image, landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Update bounding box
            lmx = [landmark.x for landmark in landmarks.landmark]
            lmy = [landmark.y for landmark in landmarks.landmark]
            min_x, max_x, min_y, max_y = int(min(lmx)*image.shape[1]), int(max(lmx)*image.shape[1]), int(min(lmy)*image.shape[0]), int(max(lmy)*image.shape[0])
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            
            # Update hand label
            hand_label = results.multi_handedness[index].classification[0].label
            hand_label = "Right" if hand_label == "Left" else "Left"
            cv2.putText(image, hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            
            # Store current landmarks, bounding box and label for future frames
            last_landmarks = landmarks
            last_bounding_box = (min_x, min_y, max_x, max_y)
            last_hand_label = hand_label
        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks_array = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
        return landmarks_array
    else:
        # If no hand is detected, use the last detected landmarks, bounding box, and label
        if last_landmarks and last_bounding_box:
            mp_drawing.draw_landmarks(image, last_landmarks, mp_hands.HAND_CONNECTIONS)
            min_x, min_y, max_x, max_y = last_bounding_box
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            if last_hand_label:
                cv2.putText(image, last_hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return None

def landmarks_to_heatmap(landmarks, img_size=(128, 128), blob_size=5):
    heatmap = np.zeros(img_size)
    for landmark in landmarks:
        x, y = int(landmark[0] * img_size[0]), int(landmark[1] * img_size[1])
        cv2.circle(heatmap, (x, y), blob_size, (255, 255, 255), -1)
    return heatmap

def preprocess_image(img, img_size=(128, 128)):
    landmarks = extract_hand_landmarks(img)
    if landmarks:
        heatmap = landmarks_to_heatmap(landmarks, img_size)
        return heatmap / 255.0
    return None

feature_extractor = load_model("cnn_model_3_mediapipe_90.h5")
dt_classifier = joblib.load('dt_classifier_3_mediapipe_90.pkl')
unique_labels = joblib.load('unique_labels_3_mediapipe_90.pkl')
img_size = (128, 128)
path = "D:/B Tech/IV Year/project/project new/dataset_mediapipe"
def generate():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
       # resized_frame = cv2.resize(frame, (int(frame.shape[1]*1.5), int(frame.shape[0]*1.5)))
        processed_img = preprocess_image(frame, img_size)
    
        if processed_img is not None:
            processed_img_reshaped = processed_img.reshape((-1, img_size[0], img_size[1], 1))
            features = feature_extractor.predict(processed_img_reshaped)
            prediction = dt_classifier.predict(features)
            predicted_gesture = unique_labels[prediction[0]]
            final_output = os.path.basename(predicted_gesture)
            text_position = (int(frame.shape[1] * 0.05), int(frame.shape[0] * 0.05))
            cv2.putText(frame, final_output, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)

        _, jpeg = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n')
@app.route('/get_gesture')
def get_gesture():
    # Your gesture recognition logic here
    recognized_gesture = "gesture_name"  # Replace with actual recognized gesture
    return jsonify(gesture=recognized_gesture)

@app.route('/video_feed')
def video_feed():
    return Response(generate(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template('index2.html')

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)
